In [161]:
import requests
import numpy as np
import pandas as pd
import sklearn.ensemble as sk
import sklearn
import statsmodels.api as sm
import matplotlib.pyplot as plt
%matplotlib inline
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [146]:
def clean_cabin(x):
    try:
        return x[0]
    except TypeError:
        return 'None'
X['Cabin']=X.Cabin.apply(clean_cabin)

In [147]:
categorical_variables=['Sex','Cabin','Embarked']
for variable in categorical_variables:
    X[variable].fillna('Missing',inplace=True)
    dummies=pd.get_dummies(X[variable],prefix=variable)
    X=pd.concat([X,dummies],axis=1)
    X.drop([variable],axis=1,inplace=True)

In [152]:
X=pd.read_csv('test.csv')

X['Age'].fillna(X.Age.mean(),inplace=True)

X['Cabin']=X.Cabin.apply(clean_cabin)

categorical_variables=['Sex','Cabin','Embarked']
for variable in categorical_variables:
    X[variable].fillna('Missing',inplace=True)
    dummies=pd.get_dummies(X[variable],prefix=variable)
    X=pd.concat([X,dummies],axis=1)
    X.drop([variable],axis=1,inplace=True)
    
X=X.drop(['PassengerId','Name','Ticket'],axis=1)

1. Creat three train data subsets: (1) with all features (2) without age (3) without age and Fare
===

In [162]:
train=pd.read_csv('train.csv')
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35,0,0,373450,8.0500,NaN,S


In [97]:
train1=train[['PassengerId','Pclass','Sex','Age','SibSp','Parch','Fare','Cabin','Embarked','Survived']].copy()
train2=train[['PassengerId','Pclass','Sex','SibSp','Parch','Fare','Cabin','Embarked','Survived']].copy()
train3=train[['PassengerId','Pclass','Sex','SibSp','Parch','Cabin','Embarked','Survived']].copy()
train1=train1.dropna()
train2=train2.dropna()
train3=train3.dropna()
print train1.shape
print train2.shape
print train3.shape
print train.shape

(712, 9)
(889, 8)
(889, 7)
(891, 12)


In [98]:
sex_dic={'male':0,'female':1}
embarked_dic={'S':0,'C':1,'Q':2}
def rename(df):
    df['Sex']=map(lambda x: sex_dic[x],df['Sex'])
    df['Embarked']=map(lambda x: embarked_dic[x],df['Embarked'])
    return df
train1=rename(train1)
train2=rename(train2)
train3=rename(train3)

In [99]:
train1.head()

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Survived
0,1,3,0,22,1,0,7.2500,0,0
1,2,1,1,38,1,0,71.2833,1,1
2,3,3,1,26,0,0,7.9250,0,1
3,4,1,1,35,1,0,53.1000,0,1
4,5,3,0,35,0,0,8.0500,0,0


2. Import test data and split into three subsets: test1,test2,test3
===

In [163]:
test=pd.read_csv('test.csv')
test_sub=test[['PassengerId','Pclass','Sex','Age','SibSp','Parch','Fare','Embarked']].copy()
print test_sub.shape
test_sub.head()

(418, 8)


,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,892,3,male,34.5,0,0,7.8292,Q
1,893,3,female,47.0,1,0,7.0000,S
2,894,2,male,62.0,0,0,9.6875,Q
3,895,3,male,27.0,0,0,8.6625,S
4,896,3,female,22.0,1,1,12.2875,S


In [101]:
test1=test_sub
test1=test1.dropna()
test1=rename(test1)
print test1.shape
test1.head()

/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


(331, 8)


,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,892,3,0,34.5,0,0,7.8292,2
1,893,3,1,47.0,1,0,7.0000,0
2,894,2,0,62.0,0,0,9.6875,2
3,895,3,0,27.0,0,0,8.6625,0
4,896,3,1,22.0,1,1,12.2875,0


In [102]:
test_nan=test_sub[test_sub.isnull().any(axis=1)]
test_nan.shape

(87, 8)

In [103]:
test2=test_nan.drop('Age',axis=1)
test2=test2.dropna()
test2=rename(test2)
test2.shape

(86, 7)

In [104]:
test3=test_nan.drop('Age',axis=1)
test3=test3[test3.isnull().any(axis=1)]
test3=rename(test3)
test3=test3.drop('Fare',axis=1)
test3.shape

(1, 6)

In [105]:
print test.shape[0]==test1.shape[0]+test2.shape[0]+test3.shape[0]

True


3. Perform random forest on train data and predict
===

In [106]:
def random_forest(df,n_estimators=500,oob_score=True):
    clf=sk.RandomForestClassifier(n_estimators=n_estimators,oob_score=oob_score)
    train_data=df[df.columns[1:-1]]
    train_truth=df['Survived']
    model=clf.fit(train_data,train_truth)
    print clf.oob_score_
    
    fi = enumerate(clf.feature_importances_)
    cols = train_data.columns
    features=pd.DataFrame([(value,cols[i]) for (i,value) in fi if value > 0.0])
    features.columns=['Importance','Feature']
    features=features.sort(['Importance'],ascending=False).reset_index()
    print features
    
    return clf

(1) On train1 and test1
==

In [107]:
clf=random_forest(train1)

0.789325842697
   index  Importance   Feature
0      2    0.288520       Age
1      5    0.259571      Fare
2      1    0.248945       Sex
3      0    0.091573    Pclass
4      3    0.047312     SibSp
5      4    0.038602     Parch
6      6    0.025477  Embarked


In [108]:
test1['Survived']=clf.predict(test1[test1.columns[1:]])

/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [109]:
test1.head()

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Survived
0,892,3,0,34.5,0,0,7.8292,2,0
1,893,3,1,47.0,1,0,7.0000,0,0
2,894,2,0,62.0,0,0,9.6875,2,0
3,895,3,0,27.0,0,0,8.6625,0,1
4,896,3,1,22.0,1,1,12.2875,0,0


(2) On train2 and test 2
==

In [110]:
clf=random_forest(train2)

0.799775028121
   index  Importance   Feature
0      4    0.413371      Fare
1      1    0.317129       Sex
2      0    0.103026    Pclass
3      2    0.063702     SibSp
4      3    0.061489     Parch
5      5    0.041282  Embarked


In [111]:
test2['Survived']=clf.predict(test2[test2.columns[1:]])

In [112]:
test2.head()

,PassengerId,Pclass,Sex,SibSp,Parch,Fare,Embarked,Survived
10,902,3,0,0,0,7.8958,0,0
22,914,1,1,0,0,31.6833,0,1
29,921,3,0,2,0,21.6792,1,0
33,925,3,1,1,2,23.4500,0,0
36,928,3,1,0,0,8.0500,0,0


(3) On train3 and test3
===

In [113]:
clf=random_forest(train3)

0.802024746907
   index  Importance   Feature
0      1    0.504801       Sex
1      0    0.209622    Pclass
2      3    0.113121     Parch
3      2    0.104447     SibSp
4      4    0.068009  Embarked


In [114]:
test3['Survived']=clf.predict(test3[test3.columns[1:]])

In [115]:
test3.head()

,PassengerId,Pclass,Sex,SibSp,Parch,Embarked,Survived
152,1044,3,0,0,0,0,0


(4) Combine results
===

In [124]:
result1=test1[['PassengerId','Survived']].copy()
result2=test2[['PassengerId','Survived']].copy()
result3=test3[['PassengerId','Survived']].copy()
frames=[result1,result2,result3]
results=pd.concat(frames)

In [125]:
results.shape

(418, 2)

In [127]:
results.head()

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,1
4,896,0


In [128]:
results.to_csv('random_forest_result.csv',header=True,index=False)